<img src="./images/DLI_Header.png" width=400/>

# Fundamentals of Accelerated Data Science # 

## 04 - Logistic Regression ##

**Table of Contents**
<br>
This notebook uses GPU-accelerated logistic regression to predict infection risk based on features of our population members. This notebook covers the below sections: 
1. [Environment](#Environment)
2. [Load Data](#Load-Data)
3. [Logistic Regression](#Logistic-Regression)
    * [Viewing the Regression](#Viewing-the-Regression)
    * [Estimate Probability of Infection](#Estimate-Probability-of-Infection)
4. [Model Explainability](#Model-Explainability)
    * [Show Infection Prevalence is Related to Age](#Show-Infection-Prevalence-is-Related-to-Age)
    * [Exercise #1 - Show Infection Prevalence is Related to Sex](#Exercise-#1---Show-Infection-Prevalence-is-Related-to-Sex)
5. [Making Predictions with Separate Training and Testing Data](#Making-Predictions-with-Separate-Training-and-Test-Data)
    * [Exercise #2 - Fit Logistic Regression Model Using Training Data](#Exercise-#2---Fit-Logistic-Regression-Model-Using-Training-Data)
    * [Use Test Data to Validate Model](#Use-Test-Data-to-Validate-Model)

## Environment ##

In [1]:
import cudf
import cuml

import cupy as cp

## Load Data

In [2]:
gdf = cudf.read_csv('./data/clean_uk_pop_full.csv', usecols=['age', 'sex', 'infected'])

In [3]:
gdf.dtypes

age         float64
sex         float64
infected    float64
dtype: object

In [4]:
gdf.shape

(58479894, 3)

In [5]:
gdf.head()

,age,sex,infected
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


## Logistic Regression ##
Logistic regression can be used to estimate the probability of an outcome as a function of some (assumed independent) inputs. In our case, we would like to estimate infection risk based on population members' age and sex.

Below we train a logistic regresion model. We first create a cuML logistic regression instance `logreg`. The `logreg.fit` method takes 2 arguments: the model's independent variables *X*, and the dependent variable *y*. Fit the `logreg` model using the `gdf` columns `age` and `sex` as *X* and the `infected` column as *y*.

In [6]:
logreg = cuml.LogisticRegression()
logreg.fit(gdf[['age', 'sex']], gdf['infected'])

LogisticRegression()

### Viewing the Regression ###
After fitting the model, we could use `logreg.predict` to estimate whether someone has more than a 50% chance to be infected, but since the virus has low prevalence in the population (around 1-2%, in this data set), individual probabilities of infection are well below 50% and the model should correctly predict that no one is individually likely to have the infection.

However, we also have access to the model coefficients at `logreg.coef_` as well as the intercept at `logreg.intercept_`. Both of these values are cuDF Series. 

Below we view these values. Notice that changing sex from 0 to 1 has the same effect via the coefficients as changing the age by ~48 years.

In [7]:
type(logreg.coef_)

cudf.core.dataframe.DataFrame

In [8]:
type(logreg.intercept_)

cudf.core.series.Series

In [9]:
logreg_coef = logreg.coef_
logreg_int = logreg.intercept_

print("Coefficients: [age, sex]")
print([logreg_coef[0], logreg_coef[1]])

print("Intercept:")
print(logreg_int[0])

Coefficients: [age, sex]
[0    0.014861
Name: 0, dtype: float64, 0    0.695666
Name: 1, dtype: float64]
Intercept:
-5.222369426308708


### Estimate Probability of Infection ###
As with all logistic regressions, the coefficients allow us to calculate the logit for each; from that, we can calculate the estimated percentage risk of infection. 

**Note**: Remembering that a 1 indicates 'infected', we assign that class' probability to a new column in the original dataframe. 

In [10]:
class_probs = logreg.predict_proba(gdf[['age', 'sex']])
class_probs

,0,1
0,0.994634,0.005366
1,0.994634,0.005366
2,0.994634,0.005366
3,0.994634,0.005366
4,0.994634,0.005366
...,...,...
58479889,0.960428,0.039572
58479890,0.960428,0.039572
58479891,0.960428,0.039572
58479892,0.960428,0.039572


In [11]:
gdf['risk'] = class_probs[1]

In [12]:
gdf.head()

,age,sex,infected,risk
0,0.0,0.0,0.0,0.005366
1,0.0,0.0,0.0,0.005366
2,0.0,0.0,0.0,0.005366
3,0.0,0.0,0.0,0.005366
4,0.0,0.0,0.0,0.005366


Looking at the original records with their new estimated risks, we can see how estimated risk varies across individuals.

In [16]:
gdf.take(cp.random.choice(gdf.shape[0], size=5, replace=False))

,age,sex,infected,risk
44393213,43.0,1.0,0.0,0.020081
23055521,61.0,0.0,0.0,0.013179
12956175,35.0,0.0,0.0,0.008993
12516497,33.0,0.0,0.0,0.008732
30413624,4.0,1.0,0.0,0.011348


## Model Explainability ##
Model explainability refers to the ability to understand and explain the decisions and reasoning underlying the predictions from machine learning models. It can be achieved by investigating how the feature variables are related to the target variable. 

### Show Infection Prevalence is Related to Age ###
The positive coefficient on age suggests that the virus is more prevalent in older people, even when controlling for sex.

For this exercise, show that infection prevalence has some relationship to age by printing the mean `infected` values for the oldest and youngest members of the population when grouped by age:

In [17]:
# %load solutions/risk_by_age
age_groups = gdf[['age', 'infected']].groupby(['age'])
print(age_groups.mean().head())
print(age_groups.mean().tail())


      infected
age           
66.0  0.020700
71.0  0.021292
64.0  0.020675
77.0  0.022102
82.0  0.022929
      infected
age           
33.0  0.015707
76.0  0.021928
74.0  0.021807
79.0  0.022518
86.0  0.023417


### Exercise #1 - Show Infection Prevalence is Related to Sex ###
Similarly, the positive coefficient on sex suggests that the virus is more prevalent in people with sex = `1` (females), even when controlling for age.

**Instructions**: <br>
* Modify the `<FIXME>` only and execute the below cell to show that infection prevalence has some relationship to sex by printing the mean `infected` values for the population when grouped by sex. .

In [18]:
sex_groups = gdf[['sex', 'infected']].groupby(['sex'])
sex_groups.mean()

,infected
sex,
0.0,0.010140
1.0,0.020713


Click ... for solution. 

## Making Predictions with Separate Training and Test Data ##
The typical process involves training the model on the training set, then using the test set to evaluate its performance. This provides a more realistic assessment of how well the model will perform on new, unseen data in real-world applications. By testing on a separate dataset, you can detect if your model is **overfitting** to the training data. Overfitting occurs when a model performs well on training data but poorly on new data. In many cases, you don't have access to truly new data, so splitting your existing data simulates this scenario. 

cuML gives us a simple method for producing paired training/testing data:

In [19]:
X_train, X_test, y_train, y_test  = cuml.train_test_split(gdf[['age', 'sex']], gdf['infected'], train_size=0.9)

In [21]:
y_train

41262001    0.0
51339764    0.0
34515490    0.0
647534      0.0
42208927    0.0
           ... 
15451473    0.0
12846561    0.0
35646641    0.0
52458379    0.0
20499871    0.0
Name: infected, Length: 52631904, dtype: float64

### Exercise #2 - Fit Logistic Regression Model Using Training Data ###

**Instructions**: <br>
* Execute the below cell to create a new logistic regression model `logreg`
* Modify the `<FIXME>` only and execute the cell below to fit the new model with the *X* and *y* training data just created.

In [22]:
logreg = cuml.LogisticRegression()

In [23]:
logreg.fit(X_train, y_train)

LogisticRegression()

Click ... for solution. 

### Use Test Data to Validate Model ###
We can now use the same procedure as above to predict infection risk using the test data:

In [24]:
y_test_pred = logreg.predict_proba(X_test, convert_dtype=True)[1]
y_test_pred.index = X_test.index
y_test_pred

2141054     0.005771
39335703    0.016365
6673843     0.006994
16655259    0.010416
22454712    0.012987
              ...   
56703753    0.033825
39933039    0.016851
21614316    0.012427
27042078    0.016421
43542884    0.019219
Name: 1, Length: 5847990, dtype: float64

As we saw before, very few people are actually infected in the population, even among the highest-risk groups. As a simple way to check our model, we split the test set into above-average predicted risk and below-average predicted risk, then observe that the prevalence of infections correlates closely to those predicted risks.

In [27]:
test_results = cudf.DataFrame()
test_results['age'] = X_test['age']
test_results['sex'] = X_test['sex']
test_results['infected'] = y_test
test_results['predicted_risk'] = y_test_pred

test_results['high_risk'] = test_results['predicted_risk'] > test_results['predicted_risk'].mean()

risk_groups = test_results.groupby('high_risk')
# test_results.head()
risk_groups.mean()

,age,sex,infected,predicted_risk
high_risk,,,,
False,29.497952,0.252741,0.010060,0.010317
True,56.188058,0.890291,0.023746,0.023323


Finally, in a few milliseconds, we can do a two-tier analysis by sex and age:

In [28]:
%%time
s_groups = test_results[['sex', 'age', 'infected', 'predicted_risk']].groupby(['sex', 'age'])
s_groups.mean()

CPU times: user 307 ms, sys: 60.5 ms, total: 367 ms
Wall time: 366 ms


infected  predicted_risk
sex age                           
0.0 6.0   0.003278        0.005857
    1.0   0.000884        0.005440
    14.0  0.005983        0.006593
1.0 5.0   0.005729        0.011507
0.0 60.0  0.013353        0.012987
...            ...             ...
    35.0  0.011654        0.008989
    13.0  0.006228        0.006496
    32.0  0.011002        0.008600
    19.0  0.007997        0.007098
1.0 12.0  0.010799        0.012754

[182 rows x 2 columns]

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** Let's move to the [next notebook](2-05_knn.ipynb). 

<img src="./images/DLI_Header.png" width=400/>